In [1]:
import numpy as np
import cv2
#import os
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm
import random
#import math

In [2]:
# calculate frechet inception distance
def calculate_fid(act1, act2):
    # calculate mean and covariance statistics
    mu1 = act1.mean(axis=0)
    sigma1 = np.cov(act1, rowvar=False)
    mu2 = act2.mean(axis=0)
    sigma2 = np.cov(act2, rowvar=False)
    
    # calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2)**2.0)
    
    # calculate sqrt of product between cov
    covmean = sqrtm(sigma1.dot(sigma2))
    
    # check and correct imaginary numbers from sqrt
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    
    # calculate score
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

In [57]:
image = np.float64(cv2.imread('../Image pre-processing/voice_to_face_data/2d_photos/ADAPT128/140012.jpg',-1))
image2 = np.float64(cv2.imread('../Image pre-processing/voice_to_face_data/2d_photos/ADAPT128/140017.jpg',-1))
image3 = np.float64(cv2.imread('../Image pre-processing/voice_to_face_data/2d_photos/ADAPT128/140018.jpg',-1))
image4 = np.float64(cv2.imread('../Image pre-processing/voice_to_face_data/2d_photos/ADAPT128/140022.jpg',-1))
image5 = np.float64(cv2.imread('../Image pre-processing/voice_to_face_data/2d_photos/ADAPT128/140023.jpg',-1))
image6 = np.float64(cv2.imread('../Image pre-processing/voice_to_face_data/2d_photos/ADAPT128/140025.jpg',-1))
image7 = np.float64(cv2.imread('../Image pre-processing/voice_to_face_data/2d_photos/ADAPT128/140027.jpg',-1))
image8 = np.float64(cv2.imread('../Image pre-processing/voice_to_face_data/2d_photos/ADAPT128/140028.jpg',-1))
D = np.zeros((8,128,128))
D[0] = image
D[1] = image2
D[2] = image3
D[3] = image4
D[4] = image5
D[5] = image6
D[6] = image7
D[7] = image8
img = np.ones((128,128))*255
img[5:,:] = image[:123,:]
img2 = np.ones((128,128))*255
img2[5:,:] = image2[:123,:]
img3 = np.ones((128,128))*255
img3[5:,:] = image3[:123,:]
img4 = np.ones((128,128))*255
img4[5:,:] = image4[:123,:]
img5 = np.ones((128,128))*255
img5[5:,:] = image5[:123,:]
img6 = np.ones((128,128))*255
img6[5:,:] = image6[:123,:]
img7 = np.ones((128,128))*255
img7[5:,:] = image7[:123,:]
img8 = np.ones((128,128))*255
img8[5:,:] = image8[:123,:]


In [4]:
calculate_fid(img,image3)


24918.19075209761

In [45]:
def lineup(generated, real_id, numFaces=3, dictionary=D):
    
    dict_size = dictionary.shape[0] #total number of face_ids
    # Set up array of face_ids for the lineup
    line_up = np.zeros(numFaces,dtype='int')
    line_up[0] = real_id # the passed face will be first in line for simplicity
    
    # Generate a randomly ordered list of all possible face_ids
    numbers = list(range(dict_size))
    random.shuffle(numbers)
    # Fill the lineup with random faces
    for i in range(line_up.shape[0]-1):
        temp = numbers.pop()
        # Checking to make sure that the passed face is not represented twice
        if (temp == real_id):
            temp = numbers.pop()
        line_up[i+1] = temp
        
    # Load the full faces for comparison using FID 
    faces = np.zeros((dictionary.shape[1],dictionary.shape[2],numFaces))
    for i in range(line_up.shape[0]):
        faces[:,:,i] = dictionary[line_up[i]]

    # Calculate the FID between each real face and the generated face
    errors = np.zeros(numFaces)
    for i in range(numFaces):
        errors[i] = calculate_fid(generated, faces[:,:,i])
    
    # minimum FID means the generated face is closest to real face
    order = np.argsort(errors)
    #change from 0-indexing to 1-indexing
    result = order[0]+1
    return result, errors[0]

In [56]:
lineup(img8, 7, 7)

(1, 1746.7000608478834)

In [15]:
gen = []
rid = [0,1,2,3,4,5,6,7]
gen.append(img)
gen.append(img2)
gen.append(img3)
gen.append(img4)
gen.append(img5)
gen.append(img6)
gen.append(img7)
gen.append(img8)

8

In [23]:
def evaluate (generated, real_id, numFaces=3):
    num_faces = len(generated)
    ranks = []
    FID_errors = []
    for i in range(num_faces):
        rank, err = lineup(generated[i], real_id[i], numFaces)
        ranks.append(rank)
        FID_errors.append(err)
    return ranks, FID_errors
        

In [55]:
r, f = evaluate(gen, rid, 3)

In [54]:
# Calculate metrics
num_samples = len(r)
# top 1
t1 = r.count(1)
# top 2
t2 = t1 + r.count(2)
# top 3
t3 = t2 + r.count(3)
print("Top 1 suspect: " + str((t1/num_samples)*100) + "%")
print("Top 2 suspect: " + str((t2/num_samples)*100) + "%")
print("Top 3 suspect: " + str((t3/num_samples)*100) + "%")

# Error metrics
# Average Error
print("Average error: " + str(sum(f)/num_samples))
# Max Error
print("maximum error: " + str(max(f)))
# Min Error
print("minimum error: " + str(min(f)))

Top 1 suspect: 100.0%
Top 2 suspect: 100.0%
Top 3 suspect: 100.0%
Average error: 389.2541773530024
maximum error: 1746.7000608478834
minimum error: 118.48598211047951
